In [3]:
# ! pip install --upgrade tensorflow

  Using cached tensorflow-2.16.1-cp39-cp39-win_amd64.whl.metadata (3.5 kB)
  Using cached tensorflow_intel-2.16.1-cp39-cp39-win_amd64.whl.metadata (5.0 kB)
  Using cached tensorboard-2.16.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.3.3-py3-none-any.whl.metadata (5.7 kB)
Using cached tensorflow-2.16.1-cp39-cp39-win_amd64.whl (2.1 kB)
Using cached tensorflow_intel-2.16.1-cp39-cp39-win_amd64.whl (376.9 MB)
Using cached keras-3.3.3-py3-none-any.whl (1.1 MB)
Using cached tensorboard-2.16.2-py3-none-any.whl (5.5 MB)
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.13.0
    Uninstalling tensorboard-2.13.0:
      Successfully uninstalled tensorboard-2.13.0
  Attempting uninstall: keras
    Found existing installation: keras 2.13.1
    Uninstalling keras-2.13.1:
      Successfully uninstalled keras-2.13.1
  Attempting uninstall: tensorflow-intel
    Found existing installation: tensorflow-intel 2.13.0
    Uninstalling tensorflow-intel-2.13.0:
  

  You can safely remove it manually.


In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
print(tf.__version__)

2.16.1


In [3]:
data = pd.read_csv("convert_numeric.csv")

In [5]:
data.head()

,duration,amount,installment_rate,present_residence,age,number_credits,people_liable,credit_risk,le_status,le_credit_history,...,le_employment_duration,le_gender,le_marital_status,le_other_debtors,le_property,le_other_installment_plans,le_housing,le_job,le_telephone,le_foreign_worker
0,6,1169,4,4,67,2,1,1,0,1,...,1,1,3,2,2,1,1,1,1,1
1,48,5951,2,2,22,1,1,0,2,3,...,2,0,1,2,2,1,1,1,0,1
2,12,2096,2,3,49,1,2,1,3,1,...,3,1,3,2,2,1,1,3,0,1
3,42,7882,2,4,45,1,2,1,0,3,...,3,1,3,1,0,1,0,1,0,1
4,24,4870,3,4,53,2,2,0,0,2,...,2,1,3,2,3,1,0,1,0,1
